## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из яндекс-диска

In [2]:
import os
import pandas as pd
path = r'C:\Users\Mikhail\Desktop\PracticeKaggle\Activities-analysis-main 3\kaggle_data' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

track_0.csv
track_1.csv
track_10.csv
track_100.csv
track_101.csv
track_102.csv
track_103.csv
track_104.csv
track_105.csv
track_106.csv
track_107.csv
track_108.csv
track_109.csv
track_11.csv
track_110.csv
track_111.csv
track_112.csv
track_113.csv
track_114.csv
track_115.csv
track_116.csv
track_117.csv
track_118.csv
track_119.csv
track_12.csv
track_120.csv
track_121.csv
track_122.csv
track_123.csv
track_124.csv
track_125.csv
track_126.csv
track_127.csv
track_128.csv
track_129.csv
track_13.csv
track_130.csv
track_131.csv
track_132.csv
track_133.csv
track_134.csv
track_135.csv
track_136.csv
track_137.csv
track_138.csv
track_139.csv
track_14.csv
track_140.csv
track_141.csv
track_142.csv
track_143.csv
track_144.csv
track_145.csv
track_15.csv
track_16.csv
track_17.csv
track_18.csv
track_19.csv
track_2.csv
track_20.csv
track_21.csv
track_22.csv
track_23.csv
track_24.csv
track_25.csv
track_26.csv
track_27.csv
track_28.csv
track_29.csv
track_3.csv
track_30.csv
track_31.csv
track_32.csv
track_33.

Переведём типы движений в числа

In [3]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [4]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [5]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):
    
# обрезаем начало и конец трека
    if a == -1:
        st = 1250
    else:
        st = a
            
    if b == -1:
        end = -1250
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1

    print(name)
#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return -1
    now_max = 0    
    try:
        Fs = 250
        Acc = (df.gFx[st:end] * 2 + df.gFy[st:end] * 2 + df.gFz[st:end] * 2) * 0.5
        y = Acc.values
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n/Fs
        frq = k/T # two sides frequency range
        frq = frq[:len(frq)//2] # one side frequency range

        Y = np.fft.fft(y)/n # dft and normalization
        Y = Y[:n//2]
    
        yabs = abs(Y)
        now_max = max(yabs[1:])
    except:
        now_max = 0.001

    Acc = (df.gFx[st:end] * 2 + df.gFy[st:end] * 2 + df.gFz[st:end] * 2) * 0.5

    amplitude = Acc.max() - Acc.min()
    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
    print()
    print(' ФУРЬЕ:', now_max * 100)
    return [amplitude, now_max]

Ampl = np.zeros(len(Data))
Furie = np.zeros(len(Data))

for df, target in zip(Data,Target):
    track_num = int(target.split('_')[1].split('.')[0]) # важный момент, что треки не по нумерации идут!
    btw = process_data(df, name=target)
    Ampl[track_num] = btw[0]
    Furie[track_num] = btw[1] * 100

Populating the interactive namespace from numpy and matplotlib
track_0.csv
 АМПЛИТУДА: 2.141

 ФУРЬЕ: 13.013806013719748
track_1.csv
 АМПЛИТУДА: 3.1321000000000003

 ФУРЬЕ: 10.277949369534713
track_10.csv
 АМПЛИТУДА: 14.652000000000001

 ФУРЬЕ: 42.60975157691195
track_100.csv
 АМПЛИТУДА: 0.40249999999999986

 ФУРЬЕ: 0.03163035358101282
track_101.csv
 АМПЛИТУДА: 0.7809999999999999

 ФУРЬЕ: 6.053041833419001
track_102.csv
 АМПЛИТУДА: 3.9466

 ФУРЬЕ: 11.510551519582823
track_103.csv
 АМПЛИТУДА: 6.3100000000000005

 ФУРЬЕ: 28.49394305516388
track_104.csv
 АМПЛИТУДА: 2.184

 ФУРЬЕ: 20.89947184383417
track_105.csv
 АМПЛИТУДА: 3.704

 ФУРЬЕ: 13.738715362206408
track_106.csv
 АМПЛИТУДА: 0.8737

 ФУРЬЕ: 0.38073302160786565
track_107.csv
 АМПЛИТУДА: 5.7721

 ФУРЬЕ: 31.918840787246054
track_108.csv
 АМПЛИТУДА: 4.9447

 ФУРЬЕ: 9.754569509208027
track_109.csv
 АМПЛИТУДА: nan

 ФУРЬЕ: 0.1
track_11.csv
 АМПЛИТУДА: 4.1762999999999995

 ФУРЬЕ: 61.44428034250029
track_110.csv
 АМПЛИТУДА: 3.8326

 ФУРЬЕ:

track_83.csv
 АМПЛИТУДА: 3.0344

 ФУРЬЕ: 13.862237398426611
track_84.csv
 АМПЛИТУДА: 4.938000000000001

 ФУРЬЕ: 108.59168267235357
track_85.csv
 АМПЛИТУДА: 27.873299999999997

 ФУРЬЕ: 62.819732586293384
track_86.csv
 АМПЛИТУДА: 9.912199999999999

 ФУРЬЕ: 10.608585501830103
track_87.csv
 АМПЛИТУДА: 3.79

 ФУРЬЕ: 18.180304812523516
track_88.csv
 АМПЛИТУДА: 5.132

 ФУРЬЕ: 27.851180864325197
track_89.csv
 АМПЛИТУДА: 11.1211

 ФУРЬЕ: 5.478823285809846
track_9.csv
 АМПЛИТУДА: 0.10209999999999986

 ФУРЬЕ: 0.13518177853032823
track_90.csv
 АМПЛИТУДА: 0.07699999999999996

 ФУРЬЕ: 0.739207806776217
track_91.csv
 АМПЛИТУДА: nan

 ФУРЬЕ: 0.1
track_92.csv
 АМПЛИТУДА: 0.7463

 ФУРЬЕ: 0.9672146224186245
track_93.csv
 АМПЛИТУДА: 1.48

 ФУРЬЕ: 8.926619271730186
track_94.csv
 АМПЛИТУДА: 1.6941000000000002

 ФУРЬЕ: 0.7618750519226671
track_95.csv
 АМПЛИТУДА: 3.624

 ФУРЬЕ: 12.158413428940648
track_96.csv
 АМПЛИТУДА: 11.1211

 ФУРЬЕ: 5.478823285809846
track_97.csv
 АМПЛИТУДА: 6.4905

 ФУРЬЕ: 37.8478111912

Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [6]:
def get_prediction(a, f):
    if f < 0.35:
        return 0
    if a < 4 and f > 7:
        return 1
    if a > 5:
        return 2
    if a < 4 and f <= 7:
        return 3
    if f < 1.5:
        return 5
    return 7

In [7]:
len(Data), len(Ampl), len(Target)

(146, 146, 146)

In [8]:
type(Data)

list

In [9]:
Ampl_new = []

i=0
for elem in Ampl:
    if not (math.isnan(elem) or elem == -1):
        i+=1
        Ampl_new.append(elem)
    else:
        Ampl_new.append(2)
len(Ampl_new), i

(146, 140)

Создадим таблицу, содержащую номер трека и предсказание.

In [40]:
prediction = []

count = 0
for am, fu in zip(Ampl, Furie):
    prediction.append(get_prediction(am, fu))

Res = pd.DataFrame({'track_num' : np.arange(len(Data)), 'action' : prediction})
Res.head(10)

,track_num,action
0,0,1
1,1,1
2,2,3
3,3,2
4,4,1
5,5,2
6,6,2
7,7,2
8,8,3
9,9,0


In [41]:
Res.to_csv("first_submit.csv", index=False)